### Install BELA workflow

In [ ]:
! git clone https://github.com/PyThaiNLP/MultiEL

In [1]:
! pip install multiel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 6.1 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 46.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.8/153.8 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 54.0 MB/s eta 0:00:00
     ━

In [1]:
from multiel import BELA
import os
import pandas as pd
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"]= "2"

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#set intial threshold for NER and EL to 0.01 to increase amount of candidates
bela_run = BELA(
 md_threshold=0.01,
 el_threshold=0.01, 
 checkpoint_name ="wiki", 
 device = "cuda:0",
 config_name="joint_el_mel_new",
 repo ="wannaphong/BELA"
)

### WDSQ

In [41]:
df_wdsq = pd.read_csv('workspace/kbqa/kbqa/thesis/Results_analysis/wdsq_res_init.csv')

### 4 modes:
1) 'bela_base' -- take initial question and rub BELA pipeline
2) 'bela_large_case' -- take initial question, capitalize all words in the question and run BELA pipeline
3) 'bela_large_ner' -- take initial question, capitalize just NER (use the code below) and run BELA pipeline
4) 'bela_just_ner' -- take just NER from question (use the code below) and run BELA pipeline

In [120]:
for i in tqdm(range(len(df_wdsq))):
    df_wdsq.loc[i, 'bela_just_ner'] = bela_run.process_batch([df_wdsq.loc[i, 'ner_largecase']])
#     df_wdsq.loc[i, 'bela_large_ner'] = bela_run.process_batch([df_wdsq.loc[i, 'ques_ner_largecase']])
#     ques = ' '.join([elem.capitalize() for elem in df_wdsq.loc[i, 'question'].split(' ')])
#     df_wdsq.loc[i, 'bela_large_case'] = bela_run.process_batch([ques])
#     df_wdsq.loc[i, 'bela_base'] = bela_run.process_batch([df_wdsq.loc[i, 'question']])



100%|██████████| 5622/5622 [03:21<00:00, 27.96it/s]


In [121]:
#rerank bela entities by el scores
for k in tqdm(range(len(df_wdsq))):
    inds = np.argsort(df_wdsq.loc[k, 'bela_just_ner'][0]['el_scores'])[::-1]
    new_arr =[df_wdsq.loc[k, 'bela_just_ner'][0]['entities'][i] for i in inds]
    cur_set = set()
    fin_ents = []
    for j in new_arr:
        if j not in cur_set:
            cur_set.add(j)
            fin_ents.append(j)
    df_wdsq.loc[k, 'bela_just_ner_ents'] = ', '.join(fin_ents)

100%|██████████| 5622/5622 [00:00<00:00, 7015.72it/s]


In [117]:
len(df_wdsq.query("bela_large_ner_ents == ''"))/len(df_wdsq)

0.0

In [110]:
#if empty, take another prediction
for i in range(len(df_wdsq)):
    if df_wdsq.loc[i, "bela_large_case_ents"] == '':
        print(i)
        df_wdsq.loc[i, 'bela_large_case'] = df_wdsq.loc[i, 'bela_base']
        df_wdsq.loc[i, 'bela_large_case_ents'] = df_wdsq.loc[i, 'bela_base_ents']

528
3291
5268
5519


In [113]:
#use 'bela_large_case' to retrieve 'ques_ner_largecase' and 'ner_largecase'

for i in tqdm(range(len(df_wdsq))):
    best_ent = np.argsort(df_wdsq.loc[i, 'bela_large_case'][0]['el_scores'])[::-1][0]
    start = df_wdsq.loc[i, 'bela_large_case'][0]['offsets'][best_ent]
    length = df_wdsq.loc[i, 'bela_large_case'][0]['lengths'][best_ent]
    part = ' '.join([elem.capitalize() for elem in df_wdsq.loc[i, 'question'][start:start+length].split(' ')])
    ques = df_wdsq.loc[i, 'question'][:start]+part+df_wdsq.loc[i, 'question'][start+length:]
    df_wdsq.loc[i, 'ques_ner_largecase'] = ques
    df_wdsq.loc[i, 'ner_largecase'] = part

100%|██████████| 5622/5622 [00:01<00:00, 4588.45it/s]


In [136]:
#prepare questions with BELA NER for mGENRE

for i in tqdm(range(len(df_wdsq))):
    best_ent = np.argsort(df_wdsq.loc[i, 'bela_large_case'][0]['el_scores'])[::-1][0]
    start = df_wdsq.loc[i, 'bela_large_case'][0]['offsets'][best_ent]
    length = df_wdsq.loc[i, 'bela_large_case'][0]['lengths'][best_ent]
    part = ' '.join([elem.capitalize() for elem in df_wdsq.loc[i, 'question'][start:start+length].split(' ')])
    begin = ' '.join([elem.capitalize() for elem in df_wdsq.loc[i, 'question'][:start].split(' ')])
    end = ' '.join([elem.capitalize() for elem in df_wdsq.loc[i, 'question'][start+length:].split(' ')])
    ques = begin+'[START] '+part+' [END]'+end
    df_wdsq.loc[i, 'mgenre_largecase_all'] = ques

100%|██████████| 5622/5622 [00:00<00:00, 6049.48it/s]


In [25]:
def topk_accuracy(df, col):
    
    count_1 = 0
    count_2 = 0
    count_3 = 0
    count_4 = 0
    count_5 = 0
    count_6 = 0
    for k in range(len(df)):
        
        if df.loc[k, col].split(', ')[0] == df.loc[k, 'subject']:
            count_1 += 1
            count_2 += 1
            count_3 += 1
            count_4 += 1
            count_5 += 1
            count_6 += 1
        elif len(df.loc[k, col].split(', ')) >= 2 and df.loc[k, col].split(', ')[1] == df.loc[k, 'subject']:
            count_2 += 1
            count_3 += 1
            count_4 += 1
            count_5 += 1
            count_6 += 1
        elif len(df.loc[k, col].split(', ')) >= 3 and df.loc[k, col].split(', ')[2] == df.loc[k, 'subject']:
            count_3 += 1
            count_4 += 1
            count_5 += 1
            count_6 += 1   
        elif len(df.loc[k, col].split(', ')) >= 4 and df.loc[k, col].split(', ')[3] == df.loc[k, 'subject']:
            count_4 += 1
            count_5 += 1
            count_6 += 1   
        elif len(df.loc[k, col].split(', ')) >= 5 and df.loc[k, col].split(', ')[4] == df.loc[k, 'subject']:
            count_5 += 1
            count_6 += 1
        elif len(df.loc[k, col].split(', ')) >= 6 and df.loc[k, col].split(', ')[5] == df.loc[k, 'subject']:
            count_6 += 1
                
    print('Top-1 accuracy:', count_1/len(df))
    print('Top-2 accuracy:', count_2/len(df))
    print('Top-3 accuracy:', count_3/len(df))
    print('Top-4 accuracy:', count_4/len(df))
    print('Top-5 accuracy:', count_5/len(df))
    print('Top-6 accuracy:', count_6/len(df))

In [46]:
topk_accuracy(df_wdsq, 'bela_base_ents')

Top-1 accuracy: 0.5597652081109925
Top-2 accuracy: 0.6561721807186055
Top-3 accuracy: 0.6860547847741018
Top-4 accuracy: 0.6997509782995376
Top-5 accuracy: 0.7072216293134116
Top-6 accuracy: 0.7127356812522234


In [112]:
topk_accuracy(df_wdsq, 'bela_large_case_ents')

Top-1 accuracy: 0.6463891853432941
Top-2 accuracy: 0.7394165777303451
Top-3 accuracy: 0.7689434364994664
Top-4 accuracy: 0.7801494130202775
Top-5 accuracy: 0.7858413376022768
Top-6 accuracy: 0.7879758093205265


In [119]:
topk_accuracy(df_wdsq, 'bela_large_ner_ents')


Top-1 accuracy: 0.6604411241551049
Top-2 accuracy: 0.7427961579509071
Top-3 accuracy: 0.7696549270722163
Top-4 accuracy: 0.7822838847385272
Top-5 accuracy: 0.7881536819637139
Top-6 accuracy: 0.7911775168979011


In [123]:
topk_accuracy(df_wdsq, 'bela_just_ner_ents')


Top-1 accuracy: 0.5962290999644255
Top-2 accuracy: 0.6536819637139808
Top-3 accuracy: 0.6673781572394166
Top-4 accuracy: 0.6725364638918534
Top-5 accuracy: 0.675382426182853
Top-6 accuracy: 0.6768054073283529


In [138]:
df_wdsq.to_csv('workspace/kbqa/kbqa/thesis/ent_linking_res/bela_for_mgenre_ques.csv', index=False)

### RuBQ

In [141]:
df_rubq = pd.read_csv('workspace/kbqa/kbqa/thesis/ent_linking_res/rubq_for_bela.csv')

### swith between russian and enligsh translation of the question

In [203]:
for i in tqdm(range(len(df_rubq))):
    df_rubq.loc[i, 'bela_just_ner'] = bela_run.process_batch([df_rubq.loc[i, 'ner_largecase']])
#     df_rubq.loc[i, 'bela_large_ner'] = bela_run.process_batch([df_rubq.loc[i, 'ques_ner_largecase']])
#     ques = ' '.join([elem.capitalize() for elem in df_rubq.loc[i, 'question_ru'].split(' ')])
#     df_rubq.loc[i, 'bela_large_case_ru'] = bela_run.process_batch([ques])
#     df_rubq.loc[i, 'bela_base_ru'] = bela_run.process_batch([df_rubq.loc[i, 'question_ru']])

100%|██████████| 1754/1754 [01:00<00:00, 28.80it/s]


In [204]:
for k in tqdm(range(len(df_rubq))):
    inds = np.argsort(df_rubq.loc[k, 'bela_just_ner'][0]['el_scores'])[::-1]
    new_arr =[df_rubq.loc[k, 'bela_just_ner'][0]['entities'][i] for i in inds]
    cur_set = set()
    fin_ents = []
    for j in new_arr:
        if j not in cur_set:
            cur_set.add(j)
            fin_ents.append(j)
    df_rubq.loc[k, 'bela_just_ner_ents'] = ', '.join(fin_ents)

100%|██████████| 1754/1754 [00:00<00:00, 8013.36it/s]


In [199]:
#retrieve question with capitalized ner and ner

for i in tqdm(range(len(df_rubq))):
    best_ent = np.argsort(df_rubq.loc[i, 'bela_base'][0]['el_scores'])[::-1][0]
    start = df_rubq.loc[i, 'bela_base'][0]['offsets'][best_ent]
    length = df_rubq.loc[i, 'bela_base'][0]['lengths'][best_ent]
    part = ' '.join([elem.capitalize() for elem in df_rubq.loc[i, 'question'][start:start+length].split(' ')])
    ques = df_rubq.loc[i, 'question'][:start]+part+df_rubq.loc[i, 'question'][start+length:]
    df_rubq.loc[i, 'ques_ner_largecase'] = ques
    df_rubq.loc[i, 'ner_largecase'] = part

100%|██████████| 1754/1754 [00:00<00:00, 4006.32it/s]


In [ ]:
len(df_rubq.query("ner_largecase_ru == ''"))/len(df_rubq)

In [157]:
#if empty, take another prediction
for i in range(len(df_rubq)):
    if df_rubq.loc[i, "bela_large_case_ents"] == '':
        print(i)
        df_rubq.loc[i, 'bela_large_case'] = df_rubq.loc[i, 'bela_base']
        df_rubq.loc[i, 'bela_large_case_ents'] = df_rubq.loc[i, 'bela_base_ents']

1101


In [146]:
topk_accuracy(df_rubq, 'bela_base_ents')

Top-1 accuracy: 0.6824401368301026
Top-2 accuracy: 0.7639680729760547
Top-3 accuracy: 0.7833523375142531
Top-4 accuracy: 0.7924743443557583
Top-5 accuracy: 0.7953249714937286
Top-6 accuracy: 0.7970353477765109


In [159]:
topk_accuracy(df_rubq, 'bela_large_case_ents')

Top-1 accuracy: 0.6322690992018244
Top-2 accuracy: 0.7246294184720639
Top-3 accuracy: 0.7485746864310148
Top-4 accuracy: 0.758266818700114
Top-5 accuracy: 0.7605473204104903
Top-6 accuracy: 0.7622576966932725


In [202]:
topk_accuracy(df_rubq, 'bela_large_ner_ents')

Top-1 accuracy: 0.6727480045610034
Top-2 accuracy: 0.749714937286203
Top-3 accuracy: 0.7730900798175598
Top-4 accuracy: 0.7839224629418472
Top-5 accuracy: 0.7873432155074116
Top-6 accuracy: 0.7884834663625998


In [205]:
topk_accuracy(df_rubq, 'bela_just_ner_ents')

Top-1 accuracy: 0.5809578107183581
Top-2 accuracy: 0.604903078677309
Top-3 accuracy: 0.6077537058152793
Top-4 accuracy: 0.6094640820980616
Top-5 accuracy: 0.6100342075256556
Top-6 accuracy: 0.6100342075256556


In [178]:
topk_accuracy(df_rubq, 'bela_base_ru_ents')

Top-1 accuracy: 0.7696693272519954
Top-2 accuracy: 0.838654503990878
Top-3 accuracy: 0.8580387685290763
Top-4 accuracy: 0.8700114025085519
Top-5 accuracy: 0.8728620296465223
Top-6 accuracy: 0.8779931584948689


In [182]:
topk_accuracy(df_rubq, 'bela_large_case_ru_ents')

Top-1 accuracy: 0.7183580387685291
Top-2 accuracy: 0.8055872291904219
Top-3 accuracy: 0.8232611174458381
Top-4 accuracy: 0.8352337514253135
Top-5 accuracy: 0.8432155074116305
Top-6 accuracy: 0.8449258836944128


In [194]:
topk_accuracy(df_rubq, 'bela_large_ner_ru_ents')

Top-1 accuracy: 0.75769669327252
Top-2 accuracy: 0.8204104903078677
Top-3 accuracy: 0.8426453819840365
Top-4 accuracy: 0.8557582668187002
Top-5 accuracy: 0.8597491448118586
Top-6 accuracy: 0.863740022805017


In [198]:
topk_accuracy(df_rubq, 'bela_just_ner_ru_ents')

Top-1 accuracy: 0.6237172177879133
Top-2 accuracy: 0.645381984036488
Top-3 accuracy: 0.6510832383124288
Top-4 accuracy: 0.6539338654503991
Top-5 accuracy: 0.6545039908779932
Top-6 accuracy: 0.6556442417331813


### Russian+English

#### join Russian and English predictions for "bela_base mode"

In [222]:
for k in tqdm(range(len(df_rubq))):
    
    inds1 = df_rubq.loc[k, 'bela_base'][0]['el_scores']
    inds2 = df_rubq.loc[k, 'bela_base_ru'][0]['el_scores']
    inds = np.argsort(inds1+inds2)[::-1]

    ents = df_rubq.loc[k, 'bela_base'][0]['entities']+df_rubq.loc[k, 'bela_base_ru'][0]['entities']

    new_arr =[ents[i] for i in inds]

    cur_set = set()
    fin_ents = []
    for j in new_arr:
        if j not in cur_set:
            cur_set.add(j)
            fin_ents.append(j)
    df_rubq.loc[k, 'bela_base_joined'] = ', '.join(fin_ents)

100%|██████████| 1754/1754 [00:00<00:00, 5752.66it/s]


#### join Russian and English predictions for "bela_large_case mode"

In [224]:
for k in tqdm(range(len(df_rubq))):
    
    inds1 = df_rubq.loc[k, 'bela_large_case'][0]['el_scores']
    inds2 = df_rubq.loc[k, 'bela_large_case_ru'][0]['el_scores']
    inds = np.argsort(inds1+inds2)[::-1]

    ents = df_rubq.loc[k, 'bela_large_case'][0]['entities']+df_rubq.loc[k, 'bela_large_case_ru'][0]['entities']

    new_arr =[ents[i] for i in inds]

    cur_set = set()
    fin_ents = []
    for j in new_arr:
        if j not in cur_set:
            cur_set.add(j)
            fin_ents.append(j)
    df_rubq.loc[k, 'bela_large_case_joined'] = ', '.join(fin_ents)

100%|██████████| 1754/1754 [00:00<00:00, 5775.74it/s]


#### join Russian and English predictions for "bela_large_ner mode"

In [225]:
for k in tqdm(range(len(df_rubq))):
    
    inds1 = df_rubq.loc[k, 'bela_large_ner'][0]['el_scores']
    inds2 = df_rubq.loc[k, 'bela_large_ner_ru'][0]['el_scores']
    inds = np.argsort(inds1+inds2)[::-1]

    ents = df_rubq.loc[k, 'bela_large_ner'][0]['entities']+df_rubq.loc[k, 'bela_large_ner_ru'][0]['entities']

    new_arr =[ents[i] for i in inds]

    cur_set = set()
    fin_ents = []
    for j in new_arr:
        if j not in cur_set:
            cur_set.add(j)
            fin_ents.append(j)
    df_rubq.loc[k, 'bela_large_ner_joined'] = ', '.join(fin_ents)

100%|██████████| 1754/1754 [00:00<00:00, 5928.38it/s]


#### join Russian and English predictions for "bela_just_ner mode"

In [226]:
for k in tqdm(range(len(df_rubq))):
    
    inds1 = df_rubq.loc[k, 'bela_just_ner'][0]['el_scores']
    inds2 = df_rubq.loc[k, 'bela_just_ner_ru'][0]['el_scores']
    inds = np.argsort(inds1+inds2)[::-1]

    ents = df_rubq.loc[k, 'bela_just_ner'][0]['entities']+df_rubq.loc[k, 'bela_just_ner_ru'][0]['entities']

    new_arr =[ents[i] for i in inds]

    cur_set = set()
    fin_ents = []
    for j in new_arr:
        if j not in cur_set:
            cur_set.add(j)
            fin_ents.append(j)
    df_rubq.loc[k, 'bela_just_ner_joined'] = ', '.join(fin_ents)

100%|██████████| 1754/1754 [00:00<00:00, 7994.93it/s]


In [227]:
topk_accuracy(df_rubq, 'bela_base_joined')

Top-1 accuracy: 0.7668187001140251
Top-2 accuracy: 0.8574686431014823
Top-3 accuracy: 0.8797035347776511
Top-4 accuracy: 0.8928164196123147
Top-5 accuracy: 0.8973774230330672
Top-6 accuracy: 0.9007981755986317


In [228]:
topk_accuracy(df_rubq, 'bela_large_case_joined')

Top-1 accuracy: 0.7183580387685291
Top-2 accuracy: 0.822690992018244
Top-3 accuracy: 0.8517673888255416
Top-4 accuracy: 0.8665906499429875
Top-5 accuracy: 0.8734321550741163
Top-6 accuracy: 0.878563283922463


In [229]:
topk_accuracy(df_rubq, 'bela_large_ner_joined')

Top-1 accuracy: 0.7537058152793614
Top-2 accuracy: 0.8472063854047891
Top-3 accuracy: 0.8740022805017104
Top-4 accuracy: 0.8825541619156214
Top-5 accuracy: 0.8876852907639681
Top-6 accuracy: 0.8916761687571265


In [230]:
topk_accuracy(df_rubq, 'bela_just_ner_joined')

Top-1 accuracy: 0.6470923603192702
Top-2 accuracy: 0.7120866590649944
Top-3 accuracy: 0.7200684150513113
Top-4 accuracy: 0.7246294184720639
Top-5 accuracy: 0.7274800456100342
Top-6 accuracy: 0.7297605473204105
